In [30]:
from bs4 import BeautifulSoup as soup
import requests as req
import pandas as pd
main_url = 'https://www.imdb.com/search/title?title_type=feature&num_votes=25000,&genres=action&start='
base_url = req.get(main_url)
Content = base_url.content

page = soup(Content,'html.parser')

#print(soup.prettify(movies_container[0]))
div = page.find('div',{'class':'desc'})
txt = div.find('span').text
txt_str = txt.split()
page_number = txt_str[2]
_str = page_number[0]+page_number[2:]
total_pages = int(int(_str)/50)
print(total_pages," Number of pages were found")
#movies_container = page.findAll('div',{'class':'lister-item-content'})
#print(len(movies_container))

25  Number of pages were found


In [31]:
url = main_url+str(page_num)
print(url)

https://www.imdb.com/search/title?title_type=feature&num_votes=25000,&genres=action&start=1


In [ ]:
p = 0
page_num = 1
while(p<total_pages):
    next_link = main_url+str(page_num)
    next_page = req.get(next_link)
    Cont=next_page.content
    soup_page=soup(Cont,"html.parser")
    movies_container=soup_page.find_all("div",{"class":"lister-item-content"})
    page_num = page_num+50
    p = p+1

In [ ]:

 #url_reg=req.get(base_url+str(next_page)+".html")

In [43]:
DataSet = []
p = 0
page_num = 1
while(p<total_pages):
    next_link = main_url+str(page_num)
    next_page = req.get(next_link)
    Cont=next_page.content
    soup_page=soup(Cont,"html.parser")
    movies_container=soup_page.find_all("div",{"class":"lister-item-content"})
    number_of_moives = len(movies_container)
    i = 0
    while(i<number_of_moives):
        movies = {}
        m = movies_container[i]
        try:
            movies['Name'] = m.find(class_='lister-item-header').get_text(strip=True)
        except AttributeError:
            moves['Name'] = None
        try:
            movies['Certificate'] = m.find(class_='certificate').get_text(strip=True)
        except AttributeError:
            movies['Certificate'] = None
        try:
            movies['Runtime'] = m.find(class_='runtime').get_text(strip=True)
        except AttributeError:
            movies['Runtime'] = None
        try:
            movies['Genre'] = m.find(class_='genre').get_text(strip=True)
        except AttributeError:
            movies['Genre'] = None
        try:
            movies['Rating'] = m.find(class_='inline-block ratings-imdb-rating').get_text(strip=True)
        except AttributeError:
            movies['Rating'] = None
        try:
            movies['Metascore'] = m.find(class_='inline-block ratings-metascore').get_text(strip=True)
        except AttributeError:
            movies['Metascore'] = None
        try:
            votes_and_gross = m.find(class_='sort-num_votes-visible').get_text(strip=True)
            v= votes_and_gross.split('|')
            n = v[0].split(':')#
            movies['Votes'] = n[1]
        except AttributeError:
            movies['Votes'] = None
        i = i + 1
        DataSet.append(movies)
    page_num = page_num+50
    p = p+1

dataframe = pd.DataFrame(DataSet)
dataframe['Name'] = dataframe['Name'].astype(str).str[2:]
dataframe['Metascore'] = dataframe['Metascore'].astype(str).str[0:2]

In [44]:
dataframe.shape

(1250, 7)

In [45]:
col_names = ["Name","Certificate",'Runtime',"Genre","Rating",'Metascore',"Votes"]
dataframe = dataframe.reindex(columns=col_names)
dataframe.to_csv("Output.csv")
dataframe.head()

,Name,Certificate,Runtime,Genre,Rating,Metascore,Votes
0,Alita: Battle Angel(2019),PG-13,122 min,"Action, Adventure, Romance",7.6,54,"65,522"
1,Spider-Man: Into the Spider-Verse(2018),PG,117 min,"Animation, Action, Adventure",8.6,87,"132,509"
2,Black Panther(2018),PG-13,134 min,"Action, Adventure, Sci-Fi",7.3,88,"472,231"
3,Polar(2019),None,118 min,"Action, Crime, Drama",6.3,19,"44,237"
4,Aquaman(2018),PG-13,143 min,"Action, Adventure, Fantasy",7.3,55,"180,136"
